In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import numpy as np
from skimage import io, color, feature
import os
from skimage.transform import resize
def compute_hog(image):
    # Convert the image to grayscale
    gray_image = color.rgb2gray(image)
    resize_image = resize(gray_image,(40,40))

    # Compute HOG features
    hog_features, hog_image = feature.hog(resize_image,orientations=9, pixels_per_cell=(8,8),cells_per_block=(2, 2), transform_sqrt=True, visualize=True)

    return hog_features

def save_hog_vectors(dataset_path, save_path):
    # Create an empty dictionary to store HOG vectors with IDs and class labels
    hog_dict = {}

    # Loop through each class in the dataset
    for class_folder in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_folder)
        if os.path.isdir(class_path):
            # Loop through each image in the class
            for filename in os.listdir(class_path):
                if filename.endswith(".jpg") or filename.endswith(".png"):
                    # Extract the ID from the filename (adjust as needed based on your naming convention)
                    image_id = os.path.splitext(filename)[0]

                    # Load the image
                    image_path = os.path.join(class_path, filename)
                    image = io.imread(image_path)

                    # Compute HOG vector for the image
                    hog_vector = compute_hog(image)
                    #print(hog_vector)

                    # Get the class label from the folder name
                    class_label = class_folder

                    # Add the HOG vector to the dictionary with the image ID as the key
                    # The value is a tuple containing the HOG vector, class label, and image ID
                    hog_dict[image_id] = (hog_vector, class_label)

    # Save the HOG vectors dictionary to a file
    np.save(save_path, hog_dict)



# Set the path to your dataset and where you want to save the HOG vectors

dataset_path = "/content/drive/MyDrive/Colab_Notebooks/bouchra/Flower-classification/flower_images"
save_path = "/content/drive/MyDrive/Colab_Notebooks/bouchra/Flower-classification/hog_vectors_by_class1.npy"

# Call the function to compute and save HOG vectors organized by class
save_hog_vectors(dataset_path, save_path)

print("Fin de programme")



$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


In [6]:
import numpy as np

save_path = "/content/drive/MyDrive/Colab_Notebooks/bouchra/Flower-classification/hog_vectors_by_class1.npy"

# Charger le contenu du fichier .npy en autorisant le pickle
loaded_data = np.load(save_path, allow_pickle=True)

# Load the HOG vectors dictionary from the file
loaded_hog_dict = loaded_data.item()

# Imprimer une sous-section du dictionnaire
print("Available Image IDs n:", list(loaded_hog_dict.values())[:10])  # Imprimer les 10 premières valeurs

# Ou, si vous voulez imprimer toutes les clés et valeurs, vous pouvez utiliser une boucle
for key, value in loaded_hog_dict.items():
    print(f"Image ID: {key}, HOG Vector: {value}")


Output hidden; open in https://colab.research.google.com to view.

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras import Sequential, layers
from tensorflow.keras.utils import to_categorical

# Function to convert labels to numerical format using one-hot encoding
def encode_labels(labels):
    encoder = LabelEncoder()
    labels_encoded = encoder.fit_transform(labels)
    labels_onehot = to_categorical(labels_encoded)
    return labels_onehot

# Load the file containing HOG vectors and classes
save_path = "/content/drive/MyDrive/Colab_Notebooks/bouchra/Flower-classification/hog_vectors_by_class1.npy"
hog_dict = np.load(save_path, allow_pickle=True).item()

# Prepare the data
hog_features_list = []
labels_list = []

for image_id, (hog_vector, class_label) in hog_dict.items():
    hog_features_list.append(hog_vector)
    labels_list.append(class_label)

# Convert lists to numpy arrays
hog_vectors = np.array(hog_features_list)
y_encoded = encode_labels(labels_list)

# Normalize HOG vectors
scaler = MinMaxScaler()
hog_vectors_normalized = scaler.fit_transform(hog_vectors)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(hog_vectors_normalized, y_encoded, test_size=0.2, random_state=42)
# Define the CNN model
input_shape = hog_vectors_normalized.shape[1:]
model = Sequential()
model.add(layers.Dense(256, input_shape=input_shape, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define K-fold cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Iterate through folds
for fold, (train_indices, val_indices) in enumerate(kfold.split(hog_vectors_normalized, np.argmax(y_encoded, axis=1))):
    print(f"Training on fold {fold+1}...")

    # Split data into training and validation sets
    X_train, X_val = hog_vectors_normalized[train_indices], hog_vectors_normalized[val_indices]
    y_train, y_val = y_encoded[train_indices], y_encoded[val_indices]

    # Train the model
    history = model.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val))

# Save the model
model.save("/content/drive/MyDrive/Colab_Notebooks/bouchra/Flower-classification/flower_classification_model_vf.h5")

# Evaluate accuracy on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")


Training on fold 1...
Epoch 1/40
125/125 [==============================] - 4s 11ms/step - loss: 1.4885 - accuracy: 0.3389 - val_loss: 1.4073 - val_accuracy: 0.4224
Epoch 2/40
125/125 [==============================] - 1s 9ms/step - loss: 1.3602 - accuracy: 0.4366 - val_loss: 1.3390 - val_accuracy: 0.4494
Epoch 3/40
125/125 [==============================] - 1s 9ms/step - loss: 1.2828 - accuracy: 0.4855 - val_loss: 1.3291 - val_accuracy: 0.4454
Epoch 4/40
125/125 [==============================] - 1s 9ms/step - loss: 1.2061 - accuracy: 0.5218 - val_loss: 1.2539 - val_accuracy: 0.4895
Epoch 5/40
125/125 [==============================] - 1s 8ms/step - loss: 1.1187 - accuracy: 0.5649 - val_loss: 1.2828 - val_accuracy: 0.4895
Epoch 6/40
125/125 [==============================] - 1s 5ms/step - loss: 1.0277 - accuracy: 0.6117 - val_loss: 1.2234 - val_accuracy: 0.5295
Epoch 7/40
125/125 [==============================] - 1s 5ms/step - loss: 0.9481 - accuracy: 0.6373 - val_loss: 1.1473 - val_

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


32/32 [==============================] - 0s 2ms/step - loss: 4.7900e-05 - accuracy: 1.0000

Test Accuracy: 100.00%


In [13]:
#indixation
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image

dataset_path_training = "/content/drive/MyDrive/Colab_Notebooks/bouchra/Flower-classification/flower_images"


# Function to convert labels to numerical format using one-hot encoding
def encode_labels(labels):
    encoder = LabelEncoder()
    labels_encoded = encoder.fit_transform(labels)
    labels_onehot = to_categorical(labels_encoded)
    return labels_onehot, encoder

# Load the pre-trained CNN model
model_path = "/content/drive/MyDrive/Colab_Notebooks/bouchra/Flower-classification/flower_classification_model_vf.h5"
cnn_model = load_model(model_path)

# Load the file containing HOG vectors and classes for test images
print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
dataset_path_testing = "/content/drive/MyDrive/Colab_Notebooks/bouchra/Flower-classification/flower"
save_path_testing = "/content/drive/MyDrive/Colab_Notebooks/bouchra/Flower-classification/hog_vectors_by_class_test.npy"

# Call the function to compute and save HOG vectors organized by class
save_hog_vectors(dataset_path_testing, save_path_testing)
test_save_path = "/content/drive/MyDrive/Colab_Notebooks/bouchra/Flower-classification/hog_vectors_by_class_test.npy"

test_hog_dict = np.load(test_save_path, allow_pickle=True).item()

# Prepare the test data
test_hog_features_list = []
test_labels_list = []

for image_id, (test_hog_vector, test_class_label) in test_hog_dict.items():
    test_hog_features_list.append(test_hog_vector)
    test_labels_list.append(test_class_label)

# Convert lists to numpy arrays
test_hog_vectors = np.array(test_hog_features_list)
test_y_encoded, label_encoder = encode_labels(test_labels_list)

# Normalize test HOG vectors
scaler = MinMaxScaler()
test_hog_vectors_normalized = scaler.fit_transform(test_hog_vectors)

# Predict the classes of test images using the CNN model
predicted_probabilities = cnn_model.predict(test_hog_vectors_normalized)
predicted_classes = np.argmax(predicted_probabilities, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_classes)

# Use KNN to find similar images within the predicted class
knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(test_hog_vectors_normalized)

# Function to find similar images for a given image
def find_similar_images(image_vector):
    distances, indices = knn.kneighbors([image_vector])
    return distances, indices

# Function to display information about a test image along with images
def display_image_info(image_index):
    original_class = label_encoder.inverse_transform([np.argmax(test_y_encoded[image_index])])[0]
    predicted_class = predicted_labels[image_index]

    print(f"Test Image {image_index + 1}:")
    print(f"Original Class: {original_class}")
    print(f"Predicted Class: {predicted_class}")

    # Load the original test image
    img_name = list(test_hog_dict.keys())[image_index]
    img_path = f"{dataset_path_training}/{original_class}/{img_name}.jpg"

    try:
        original_img = image.load_img(img_path, target_size=(224, 224))

        # Display the original image
        plt.figure()
        plt.imshow(original_img)
        plt.title("Original Image")
        plt.axis("off")
        plt.show()

    except FileNotFoundError:
        print(f"Original image not found: {img_path}")

    # Find similar images for the given image
    distances, indices = find_similar_images(test_hog_vectors_normalized[image_index])

    print("\nSimilar Images:")
    for i, index in enumerate(indices[0]):
        similar_image_class = predicted_labels[index]

        # Load the similar image
        similar_img_name = list(test_hog_dict.keys())[index]
        similar_img_path = f"{dataset_path_testing}/{similar_image_class}/{similar_img_name}.jpg"

        try:
            similar_img = image.load_img(similar_img_path, target_size=(224, 224))

            # Display the similar image
            plt.figure()
            plt.imshow(similar_img)
            plt.title(f"Similar Image {i + 1} - Predicted Class: {similar_image_class}")
            plt.axis("off")
            plt.show()

        except FileNotFoundError:
            print(f"Similar image not found: {similar_img_path}")

        print(f"Distance: {distances[0][i]}")

    print("\n" + "=" * 50)

# Loop through all test images and display their information
for i in range(len(test_hog_vectors_normalized)):
    display_image_info(i)


Output hidden; open in https://colab.research.google.com to view.